In [1]:
import requests
from datetime import datetime
from sqlalchemy import create_engine, text
from quicknotifier import notify

In [9]:
endpoints = ["https://www.movsisyan.info"]

with open("secret4.txt") as f:
    secret4 = f.read()

engine = create_engine(
    f"mysql+mysqlconnector://{secret4}@movsisyan.info/movsisya_dashboard")

with engine.connect() as connection:
    for endpoint in endpoints:
        start = datetime.now()
        r = requests.get(endpoint)
        end = datetime.now()

        ms = int((end - start).microseconds /1000)
        print(start, r.status_code, ms, endpoint)
        connection.execute(text(
            f"INSERT INTO `uptime_log`(`time`, `status_code`, `ms`, `endpoint`) \
                VALUES ('{start.strftime('%Y/%m/%d %H:%M:%S')}','{r.status_code}','{ms}','{endpoint}')"))
        if r.status_code != 200:
            notify(f"Status code {r.status_code} while checking uptime of {endpoint}. {r.content}")
        if ms > 2000:
            notify(f"High response time {ms}ms while checking uptime of {endpoint}.")
        

2023-04-19 22:31:26.040606 200 35 https://www.movsisyan.info


In [26]:
import requests
from datetime import datetime
from sqlalchemy import create_engine, text
from sqlalchemy.exc import ResourceClosedError
from quicknotifier import notify
import os

endpoints = ["https://www.movsisyan.info"]

with open("secret.txt") as f:
    engine = create_engine(
        f"mysql+mysqlconnector://{f.read()}@movsisyan.info/movsisya_dashboard")

sql_scripts = {}
for script in os.listdir("scripts"):
    with open("scripts/" + script) as f:
        sql_scripts[script] = f.read()

with engine.connect() as connection:
    for script in sorted(sql_scripts.keys()):
        try: 
            result = connection.execute(text(sql_scripts[script]))
            data = {key: [] for key in result.keys()}
            for i, row in enumerate(result):
                [data[list(data.keys())[col]].append(value)
                for col, value in enumerate(row)]
            
            if "ms" in data.keys():
                notify(f"Latency today:\n\t\tMin\t{str(data['ms'][0]).rjust(5)}ms,\n\t\t25%" +
                       f"\t{str(data['ms'][1]).rjust(5)}ms\n\t\t50%\t{str(data['ms'][2]).rjust(5)}ms" +
                       f"\n\t\t75%\t{str(data['ms'][3]).rjust(5)}ms\n\t\tMax\t{str(data['ms'][4]).rjust(5)}ms")
        except ResourceClosedError as e:
            continue